In [1]:
!pip install -r requirements.txt

In [44]:
from persistence import Connection as Database
db: Database = Database()

Connected to DB


In [45]:
resumes = db.fetch_all_unlabeled_resumes()

In [46]:
len(resumes)

30

In [47]:
import json
resume = {}

with open("demo/ds10_modified.json") as f:
    resume = json.load(f)

In [48]:
resume['experience']

{'value': [{'value': {'company': 'Apple',
    'title': 'Data Scientist',
    'location': 'San Francisco',
    'fromDate': 'June 2022',
    'toDate': 'Present',
    'description': ''},
   'label': 0},
  {'value': {'company': 'iRobot',
    'title': 'Data Scientist',
    'location': 'Boston',
    'fromDate': 'August 2021',
    'toDate': 'May 2022',
    'description': 'Developing data mining solutions to create practical insights for optimizing Direct-to-Customer plans and activities. Improving product quality by analyzing customer and product data to identify issues and improvement areas. Built a 52% more accurate demand and sales forecasting model through better model selection and feature engineering. Producing dashboards and Decision Support Systems to help stakeholders in data-driven decision-making'},
   'label': 0},
  {'value': {'company': 'UMass Boston College of Management',
    'title': 'Graduate Research Assistant',
    'location': 'Boston',
    'fromDate': 'September 2016',
   

In [49]:
import importlib

In [50]:
job_description = ""
with open("jd_examples/DS_JD2.txt", 'r') as f:
    job_description = f.read()
    

In [51]:
from model import Job
importlib.reload(Job)
Job = Job.Job
from model.keyword_utils import extract_requirements_JD

from persistence.cloud_storage.cloud_storage import read_job_keywords
from util.json_to_df import create_df, get_resume_dict_from_dataframe

job_title = "Data Scientist"
description_keywords = extract_requirements_JD(job_description)
job_keywords_csv = read_job_keywords(job_title)
job = Job(job_title)
job.load_from_csv_data(job_keywords_csv)

orig_keywords = set(job.keywords.keys())
jd_keyword_set = set(description_keywords)
kw = list(orig_keywords.intersection(jd_keyword_set))

job.update_keywords_from_description(description_keywords)


resume_df, md = create_df(resume)

In [52]:
job.keywords.keys()

dict_keys(['machine learning', 'r', 'data science', 'python', 'deep learning', 'natural language processing', 'tableau', 'sql', 'predictive modeling', 'statistics', 'data analysis', 'data visualization', 'statistical modeling', 'tensorflow', 'predictive analytics', 'artificial intelligence', 'data mining', 'scikit-learn', 'data analytics', 'apache', 'spark', 'big data', 'hadoop', 'algorithms', 'git', 'amazon web services', 'big data analytics', 'hive', 'scala', 'matlab', 'pandas', 'numpy', 'sas', 'microsoft power bi', 'nlp', 'aws', 'ai', 'mathmatics', 'data scientist', 'b.e/b.tech/m.e/m.tech/ms', 'computer science', '3-6 years', 'advanced computer vision', 'keras', 'yolo', 'caffe', 'high dimension classifiers', 'advanced techniques', 'gcp', 'python flask\nadditional information'])

In [53]:
jd_keyword_set

{'3-6 years',
 'advanced computer vision',
 'advanced techniques',
 'b.e/b.tech/m.e/m.tech/ms',
 'caffe',
 'computer science',
 'deep learning',
 'gcp',
 'high dimension classifiers',
 'keras',
 'machine learning',
 'python',
 'python flask\nadditional information',
 'tensorflow',
 'yolo'}

In [54]:
from model import resume_scorer
# Make sure to reload everytime
importlib.reload(resume_scorer)
ResumeScorer = resume_scorer.ResumeScorer

scorer: ResumeScorer = ResumeScorer(job)
scored = scorer.score_resume_as_dataframe(resume_df)


/opt/conda/lib/python3.7/site-packages/gensim/models/keyedvectors.py:1166: RuntimeWarning: invalid value encountered in true_divide
  similarities = dot_products / (norm * all_norms)


In [55]:
importlib.reload(resume_scorer)

cut, stats = resume_scorer.shorten_resume(scored, kw)

print(len(scored))
print(len(cut))
cut[['type', 'scoring_text', 'score', 'duration', 'raw_keywords' ]]

31
14


,type,scoring_text,score,duration,raw_keywords
30,certifications,Neural Networks and Deep Learning,1.770000,NaN,[deep learning]
29,certifications,Triplebyte Certified Data Scientist,0.456379,NaN,[triplebyte certified data scientist]
1,contactInfo,Monica Music,inf,NaN,[monica music]
11,education,"Doctor of Philosophy (Ph.D.), Information Syst...",0.222112,60.0,"[information systems for, stem]"
13,education,"Bachelor of Science - BS, Mechanical Engineering",0.302949,48.0,"[bachelor of science - bs, mechanical engineer..."
2,experience,Data Scientist,1.014000,6.0,[data scientist]
3,experience,Data Scientist Developing data mining solution...,0.321525,9.0,"[data scientist developing, decision support s..."
4,experience,"Graduate Research Assistant In my research, I ...",0.301502,59.0,"[graduate research, machine learning, nltk & g..."
5,experience,Quantitative Methods Fellow Teaching courses i...,0.762447,9.0,"[quantitative methods, business analytics, dat..."
6,experience,Data Science Intern I was using the company's ...,1.666000,6.0,"[machine learning, python, sas, 42%, sql]"


In [56]:
import json
from util.json_to_df import get_resume_dict_from_dataframe


output_dict = get_resume_dict_from_dataframe(cut)
# del output_dict['_id']
print(json.dumps(output_dict))

{"contactInfo": {"address": "Chicago, IL", "website": "my-website.com", "linkedin": "linkedin.com/in/me123", "name": "Monica Music", "phoneNumber": "01012345", "email": "monica@music.co.kr", "github": "monicamusic123", "birthday": "24 December 1985", "family": "5 kids, divorced", "keywords": [["monica music", 1.0]], "raw_keywords": ["monica music"], "education_heuristic": 0.0, "experience_heuristic": 0.0, "contact_heuristic": Infinity, "interest_heuristic": 0.0, "covered_keywords": [], "keyword_cover_heuristic": 0.0, "score": Infinity}, "summary": {"value": "With a Ph.D. in data science and more than five years of working in data science and analytical roles, I have significant experience in creating actionable insights and data-driven solutions through research, experimentation, data analysis, and developing advanced analytical and machine learning models. I collaborated with different teams and managed cross- functional projects to provide innovative solutions to challenging problems

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [57]:
set(stats['missing_keywords'])

{'tensorflow'}

In [65]:
cut = cut[resume_df.columns]
cut = cut.drop(["keywords"], axis=1)

In [66]:
cut.columns

Index(['value', 'label', 'type', 'address', 'website', 'linkedin', 'name',
       'phoneNumber', 'email', 'github', 'birthday', 'family', 'company',
       'title', 'location', 'fromDate', 'toDate', 'description', 'duration',
       'time_since', 'institution', 'proficiency', 'level', 'date',
       'scoring_text', 'multiplier'],
      dtype='object')

In [59]:
set(stats['all_original_keywords'])

{'# 56',
 '42%',
 '5.8%',
 'ai',
 'ai-based voice assistant systems:',
 'approximately 450,000',
 'bachelor of science - bs',
 'bi',
 'customer relationship management',
 'data mining',
 'data scientist',
 'data scientist developing',
 "dean's fellowship award for phd program",
 'decision support systems',
 'deep learning',
 'diploma',
 'english',
 'erp',
 'gcp',
 'graduate research',
 'information systems for',
 'machine learning',
 'mathematics discipline',
 'mechanical engineering',
 'monica music',
 'more than five years',
 'national entrance exam of universities',
 'nltk & gensim',
 'persian',
 'physics',
 'poisson regression',
 'python',
 'quantitative methods, business analytics',
 'random forest',
 'rnn',
 'sas',
 'second',
 'sql',
 'sql & r',
 'statistical inference',
 'statistical learning',
 'statsmodels & sklearn & keras',
 'stem',
 'suppliers learning',
 'the interaction and trust',
 'triplebyte certified data scientist',
 'xgboost',
 'yolo'}

In [60]:
from util.json_to_df import get_resume_dict_from_dataframe
import json

output = get_resume_dict_from_dataframe(cut, md)
del output["_id"]

json.dumps(output)

print("")

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [61]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [62]:
text = "Olav, like to eat cake for my brithday, and he knows vision transformers"
doc = nlp(text)

for v in doc.ents: 
    print(v)

Olav


In [64]:
o = {**resume, "description": job_description} 

print(json.dumps(o))

{"_id": "hejhej123", "title": "Data Scientist", "contactInfo": {"address": "Chicago, IL", "website": "my-website.com", "linkedin": "linkedin.com/in/me123", "name": "Monica Music", "phoneNumber": "01012345", "email": "monica@music.co.kr", "github": "monicamusic123", "birthday": "24 December 1985", "family": "5 kids, divorced"}, "summary": {"value": "With a Ph.D. in data science and more than five years of working in data science and analytical roles, I have significant experience in creating actionable insights and data-driven solutions through research, experimentation, data analysis, and developing advanced analytical and machine learning models. I collaborated with different teams and managed cross- functional projects to provide innovative solutions to challenging problems and advise executives on operations, product, marketing, and customer experience strategies.", "label": 0}, "experience": {"value": [{"value": {"company": "Apple", "title": "Data Scientist", "location": "San Franc